In [3]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from shapely import wkb
import geopandas
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)[0]

In [5]:
# CONECTAR A AMIGOCLOUD
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [6]:
def generar_reporte(codigo_canhero):
    # EXTRAER DATOS DE CREDITO
    query_credito = {'query': 'select * from dataset_307560 where cast(SPLIT_PART(canhero, \'/\', 1) as integer) = {cod_ca}'.format(cod_ca=codigo_canhero)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_credito)
    credito_data = credito_select['data'][0]
    credito_data['fecha_informe'] = (datetime.strptime(credito_data['fecha_1ra_insp'].split(' ')[0], '%Y-%m-%d') + timedelta(days=1)).strftime('%d/%m/%Y')
    credito_data['fecha_1ra_insp'] = datetime.strptime(credito_data['fecha_1ra_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())

    # EXTRAER DATOS DE PROPIEDADES
    query_propiedades = {'query': 'select * from dataset_307562 where insp_ref_prop = \'{insp_ref_prop}\''.format(insp_ref_prop=credito.amigo_id)}
    propiedades_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_propiedades)
    propiedades_data = propiedades_select['data']

    # EXTRAER DATOS DE LOTES
    propiedades = []
    for prop in propiedades_data:
        prop['lotes'] = []
        query_lotes = {'query': 'select * from dataset_307563 where prop_ref_lote=\'{prop_ref_lote}\''.format(prop_ref_lote=prop['amigo_id'])}
        lotes_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_lotes)
        for lote_select in lotes_select['data']:
            lote = collections.namedtuple("lote", lote_select.keys())(*lote_select.values())
            prop['lotes'].append(lote)
        propiedades.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))

    # EXTRAER FOTOS
    fotos = []
    for propiedad in propiedades:
        for lote in propiedad.lotes:
            print(lote.lote, lote.amigo_id)
            query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_46845 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=lote.amigo_id)}
            fotos_filename = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_fotos)
            if fotos_filename['data'] != []:
                dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
                fotos.append(dic)

    # GENERERAR PLANOS
    i = 0
    lista_planos = []
    for propiedad in propiedades:
        i = i + 1
        lista_planos.append(generar_plano(propiedad, i))


    # GENERAR REPORTE
    doc = DocxTemplate("templates/template_word_1ra.docx")
    #generar lista de InlineImage de planos 
    lista_InlineImage = []
    for plano in lista_planos:
        lista_InlineImage.append(docxtpl.InlineImage(doc, image_descriptor=plano, width=Mm(150)))
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        file = open('fotos/' + foto['file_name'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['file_name'], width=Mm(120)), 'lote': foto['lote']})
    firma_respon = None
    if credito.respon_1ra_insp == 'Rogelio Acuña Rodríguez':
        firma_respon = docxtpl.InlineImage(doc, image_descriptor='templates/firma_rogelio.png', width=Mm(60))
    else:
        firma_respon = docxtpl.InlineImage(doc, image_descriptor='templates/firma_jaldin.png', width=Mm(60))
    context = {'credito':credito, 'propiedades':propiedades, 'planos':lista_InlineImage, 'fotos':lista_fotos_inline, 'firma':firma_respon}
    doc.render(context)
    file_name = credito.canhero.replace(' / ', '_')
    doc.save('_' + file_name + '.docx')

In [7]:
def generar_plano(propiedad, i):
    lotes_lista = []
    for lote in propiedad.lotes:
        lotes_lista.append(lote._asdict())
    df = pd.DataFrame(lotes_lista)
    df['geometria'] = df['geometria'].apply(convertir_wkb)

    #Convertir a GeoDataFrame
    data = geopandas.GeoDataFrame(df, geometry='geometria')
    #data
    #ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)

    data['coords'] = data['geometria'].apply(lambda x: x.representative_point().coords[:])
    data['coords'] = [coords[0] for coords in data['coords']]

    data.crs = "EPSG:4326"
    data = data.to_crs(epsg=3857)

    
    fig = plt.figure(i, figsize=(20,20))
    #fig = plt.figure(1)
    ax = None
    ax = fig.add_subplot()

    #data.set_crs(epsg=4326, inplace=True)

    data.apply(lambda x: ax.annotate(text=x.lote + ' \n' + str(x.superficie) + ' ha', xy=x.geometria.centroid.coords[0], ha='center', va='center', color='black', fontsize=12, weight=1000, bbox=dict(facecolor=(1,1,1,0.3), edgecolor='none', pad=0)), axis=1);

    #data.boundary.plot(ax=ax, color=(0,0,0,0), linewidth=0)

    #data = data.to_crs(epsg=3857)

    minx, miny, maxx, maxy = data.total_bounds
    ax.set_xlim(minx - 500, maxx + 500)
    ax.set_ylim(miny - 400, maxy + 400)

    data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2, figsize=(20,20))
    #data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2)
    
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
    ax.set_axis_off()
    ax.set_title(str(propiedad.propiedad), fontsize=20)
    ruta = 'planos/' + str(propiedad.amigo_id) + '.jpeg'
    fig.savefig(ruta, dpi = 300, bbox_inches='tight')
    plt.clf()
    return ruta

In [8]:
semana = 'S03'
query = {'query': 'select split_part(canhero, \' / \', 1) cod, split_part(canhero, \' / \', 2) nom from dataset_307560 where semana_1ra = \'{sem}\''.format(sem=semana)}
project = 'https://app.amigocloud.com/api/v1/projects/32672/sql'
select = amigocloud.get(project, query)
data = select['data']

In [9]:
data

[{'cod': '18027', 'nom': 'AGUILERA TARRADELLES GUILLERMO'},
 {'cod': '2408', 'nom': 'CHOQUE CACERES FREDDY'},
 {'cod': '743', 'nom': 'ARTEAGA CHAVEZ ROBERT E.'},
 {'cod': '2029', 'nom': 'COLAMARINO DI SILVIO ANGELO'},
 {'cod': '12478', 'nom': 'SERRATE COLAMARINO HUGO'},
 {'cod': '2662', 'nom': 'DON SERAFIN SRL.'},
 {'cod': '633', 'nom': 'AGROP. VICENTE ROCA GIL SRL'},
 {'cod': '41820', 'nom': 'ASUBY S.R.L.'},
 {'cod': '2392', 'nom': 'COLAMARINO SUAREZ DOMENICO A.'},
 {'cod': '40139', 'nom': 'ROCA SUAREZ MARIA ALEJANDRA'}]

In [10]:
#lista = [1068, 18062, 10739, 41955, 2392, 15162, 2517, 8706, 15085, 430, 874, 5498, 2505, 2274]
lista = [int(i['cod']) for i in data]

In [11]:
lista

[18027, 2408, 743, 2029, 12478, 2662, 633, 41820, 2392, 40139]

In [13]:
for i in lista:
    generar_reporte(i)
    print('======================', i)

L27 1fff509938fc4f52b0d70f487e9459cc
L23 cbef3ba9e1e8487dba90593c3456c315
L15.1 6bbdac1c354643d284bcfa0aa89603cb
L4 aca011bb784149eea91d65e6707b8459
L3.2 a60f957b338c4847a0a097436d6f84c8
L10.1 928c3de1d58c41ab874f6c1faff403ca
L13 74045138f8c34892943246fa43bf8877
L17.3 111661468c554f52a12b4072428d178e
L5 1db065977f454092a2865f06459650d6
L18.3 99141238f52343688853ff3957aa695d
L2 3e07c696812949e98d98256447e39e77
====================== 18027
L2.2 cf8586bba0be4592b4fdb076c27aa768
L3.2 b88a6076290b436aabd4b78dc4b5bc87
====================== 2408
L3.2 9a1f7a6ec5db4ca48b91ef0d1e38912c
L4.1 d96b5c4c5a444a5bbc39e75f9f3d385a
====================== 743
L4.2.1 41c898ca209742108d45beb1affb57a9
L4.3 836dddd44de24d7893f080874d0a8781
L3.1 237345f93d6e4191bb1b4cdc672c31c3
L1.1 197efe8280aa45a1bd7a088ad3a52a4f
L3.3 c6e14e0da67d4384ab9f5524e264bd7f
L3.4 03634173be494396b3a1c4a37f65d05d
L5.2 0d16f6dc81ca4d32a706cad47768b857
L5.1 0ebb47f725d34a83a431c20fcfd05ead
L5.4 f409391e287a4f118cafe22a46eaac59
L1.6 cd

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>